In [47]:
import pandas as pd
import shutil
import os
from openpyxl import load_workbook

In [48]:
# File paths (update these with actual file paths, where the files are located on your system)

donor_file_path = "sample_donor_metadata.xlsx" # Path to the filled out donor metadata file
bulk_file_path = "C:/Users/stouq/Downloads/bulk_import_template.xlsx" # Path to the bulk import template file

In [49]:
# Extract donor filename (without extension)
donor_filename = os.path.splitext(os.path.basename(donor_file_path))[0]

# Create output file name dynamically, so you know what each file is for or what
# collections it references
output_file_path = f"bulk_file_filled_{donor_filename}.xlsx"

In [50]:
# Load the donor metadata file
donor_df = pd.read_excel(donor_file_path)

# Load the bulk_import_template file
bulk_df = pd.read_excel(bulk_file_path, header=4)

# Make a copy of the bulk_import_template file so you don't overwrite the original
shutil.copy(bulk_file_path, output_file_path)

'bulk_file_filled_sample_donor_metadata.xlsx'

In [51]:
# Create the General column by merging the separate fields (created_by, place, people, context) in the donor metadata file
def format_general(row):
    parts = []
    if pd.notna(row.get("created_by", None)):  # Use .get() to avoid KeyError
        parts.append(f"Created by {row['created_by']}.")
    if pd.notna(row.get("place", None)):
        parts.append(f"Created in {row['place']}.")
    if pd.notna(row.get("people", None)):
        parts.append(f"People described/pictured: {row['people']}.")
    if pd.notna(row.get("context", None)):
        parts.append(f"{row['context']}.")
    if pd.notna(row.get("id_number", None)):
        parts.append(f"ID Number: {row['id_number']}")
    
    return " ".join(parts) if parts else None  # Return None if all fields are empty

# Apply the function to create the 'General' column for the bulk_import_template file
donor_df["General"] = donor_df.apply(format_general, axis=1)

In [52]:
# Define column mapping from Donor File to Bulk File
column_mapping = {
    "grouping_or_item": "Description Level",
    "title": "Title",
    "date_begin": "Date(1) Begin",
    "date_end": "Date(1) end",
    "quantity": "Extent number",
    "measurement": "Extent type: cubic feet, cds, etc.",
    "history_of_ownership": "Custodial History",
    "General": "General"
}

In [53]:
donor_df = donor_df.rename(columns=column_mapping)

# Transform grouping_or_item values (grouping and item) 
# to match the bulk import template values (series and file)
donor_df["Description Level"] = donor_df["Description Level"].replace({
    "grouping": "series",
    "item": "file"
})

# Use the Hierarchical Relationship logic to determine the level of the description
donor_df["Hierarchical Relationship"] = donor_df["Description Level"].map({
    "series": 1,
    "file": 2
})

# Add Date Type logic based on Date Begin and Date End
def determine_date_type(row):
    if pd.notna(row["Date(1) Begin"]) and pd.isna(row["Date(1) end"]):
        return "single"
    elif pd.notna(row["Date(1) Begin"]) and pd.notna(row["Date(1) end"]):
        return "inclusive"
    return None  # Return None for rows without a date begin or date end

donor_df["Date(1) Type"] = donor_df.apply(determine_date_type, axis=1)

# Load the workbook and get the active sheet to preserve formatting
wb = load_workbook(output_file_path)
ws = wb.active

# Load existing data starting from row 6 (preserving rows 1-5), since headers start on row 5
existing_data = pd.read_excel(output_file_path, header=4)

donor_df = donor_df[[col for col in bulk_df.columns if col in donor_df.columns]]

# Append new data below the existing data (so you cant overwrite any existing data 
# and can run multiple donor metadata files through at once)
updated_data = pd.concat([existing_data, donor_df], ignore_index=True)

# Write updated data starting from row 6 (after the header)
for r_idx, row in enumerate(updated_data.itertuples(), 6):  # Start writing from row 6
    for c_idx, value in enumerate(row[1:], 1):  # Skip the index (row[0])
        ws.cell(row=r_idx, column=c_idx, value=value)

# Save the updated workbook (with the preserved formatting)
wb.save(output_file_path)

print(f"Data successfully transferred. The updated file is saved as: {output_file_path}")

C:\Users\stouq\AppData\Local\Temp\ipykernel_38096\709559630.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_data = pd.concat([existing_data, donor_df], ignore_index=True)


Data successfully transferred. The updated file is saved as: bulk_file_filled_sample_donor_metadata.xlsx
